In [1]:
using JLD,JLD2,Knet
using Base.Iterators: flatten
using Random, Distributions

In [2]:
include("types.jl")
include("pre_processing.jl")
include("encoder_utils.jl")

fillvecs! (generic function with 1 method)

In [3]:
d = JLD.load("pretrained_model.jld2")
v = create_vocab(d)
corpus = load_conllu("tr_imst-ud-train.conllu",v)
wmodel = makewmodel(d)

9-element Array{Array{Float32,2},1}:
 [0.593419 0.190279 … 0.11003 -0.000173569; -0.236287 0.455614 … 0.138825 -0.0971059; … ; -0.341316 0.0835031 … -0.0285478 -0.0718531; -0.205958 -0.168918 … -0.13226 -0.0319474]
 [0.0708071 -0.456333 … 0.557484 0.094816; -1.61625 -1.37924 … 0.342474 0.548784; … ; 1.07664 -0.430524 … -0.360356 0.668587; 0.324491 -0.175173 … -0.790751 1.16417]            
 [1.2185; 1.52402; … ; -1.1745; -1.53404]                                                                                                                                        
 [1.47578 0.055072 … -0.2823 -1.58338; -1.95253 -0.402249 … -0.221281 -0.667834; … ; 1.28302 -0.0459206 … -2.12551 -0.95058; 0.924036 0.269601 … 0.515548 7.38111]               
 [-0.916012; -2.25621; … ; -0.166855; 0.169325]                                                                                                                                  
 [1.70814 0.787587 … -0.636378 -1.76298; 4.32346 5.14362 … 0.705938 -0.04

In [6]:
s = corpus[1]

Sanal parçacıklar sa bunların hiçbirini yapamazlar . 

In [5]:
wembed, wdata, wmask = fillvecs!(wmodel, corpus, v)

┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #lmloss#18(::Array{Float64,1}, ::Function, ::Array{Array{Float32,2},1}, ::Array{Array{Int64,1},1}, ::Array{Array{Float32,1},1}, ::Array{Any,1}, ::Array{Any,1}) at encoder_utils.jl:59
└ @ Main /home/caltan14/gbdp/encoder_utils.jl:59


(Float32[-2.69957e-40 -2.83062e-43 … -0.0 -0.0; -0.994933 -0.00016526 … -0.000105809 -0.00137349; … ; -5.69672e-31 -2.63217e-37 … -1.14541e-31 -5.53253e-32; -3.98312e-7 -1.51246e-8 … -0.0015989 -0.000710246], Array{Int64,1}[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2787], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2948]  …  [1, 2, 13519, 2, 2, 2, 638, 2, 13530, 2  …  1, 13784, 2, 2, 2, 2, 3910, 2, 1076, 58], [13516, 2, 3296, 2, 2, 2, 1155,

In [16]:
ev = extend_vocab!(v, corpus)

ExtendedVocab(Dict("Abr"=>26,"PCNom"=>18,"Dup"=>27,"PCIns"=>30,"Since"=>32,"Postp"=>31,"Interj"=>24,"Ques"=>14,"Demons"=>5,"NAdj"=>11…), Dict("VerbForm=Vnoun"=>43,"Person=3"=>4,"Case=Acc"=>11,"Number=Sing"=>7,"Mood=Nec"=>54,"Aspect=Prog"=>27,"Person[psor]=3"=>13,"VerbForm=Conv"=>44,"Aspect=Hab"=>15,"Voice=CauPass"=>46…), Vocab(Dict('\x00\x00\x00\x42'=>37,'\x00\x00\x20\x1d'=>116,'\x00\x00\x00\x58'=>87,'\x00\x00\x20\x44'=>110,'\x00\x00\x00\x56'=>59,'\x00\x00\x00\xd7'=>107,'\x00\x00\x00\x48'=>29,'\x00\x00\x00\x75'=>5,'\x00\x00\x00\x50'=>11,'\x00\x00\x00\x7a'=>61…), Dict("çıkaramayacaklarını"=>7805,"apartmanın"=>13565,"Güreş"=>2733,"karıştı"=>1790,"ikibin'de"=>7225,"formülü"=>4676,"ANAP'ı"=>4506,"ihmal"=>5116,"Virgülden"=>9483,"il"=>6725…), Dict("null"=>8668,"Secure"=>7932,"Puppy"=>9040,"progression"=>11757,"gathered"=>6954,"Core"=>3600,"lovers"=>8832,"underground"=>9093,"backup"=>5047,"caught"=>2501…), "<s>", "</s>", "<unk>", '\x00\x00\x00\x12', '\x00\x00\x00\x13', '\x00\x00\x00\x11', Dic

In [20]:
mutable struct FeatureSource
    postags::Array{Float64}
    xpostags::Array{Float64}
    feats::Array{Float64}
end

In [21]:
Random.seed!(5)

MersenneTwister(UInt32[0x00000005], Random.DSFMT.DSFMT_state(Int32[-85655800, 1072952617, -522203688, 1073721902, 1588056292, 1072823262, 1930843968, 1073164255, -719121749, 1073022455  …  647343590, 1073165527, 1405924594, 1072769931, 1969171087, 42945320, -226019479, -1469955508, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000000

In [22]:
# Size of the randomly initialized column vectors that represents features such that xpostag, upostag, feats
tagrepsize = 32

32

In [24]:
# Creates column vectors for all unique xpostags, postags, and features; acts like a vocab, extendedvocab
function createcolvecs(corpus,ev)
    dist = Normal()
    xpostags = rand(dist, (tagrepsize, length(keys(ev.xpostags))))
    postags = rand(dist, (tagrepsize, length(keys(ev.vocab.postags))))
    feats = rand(dist, (tagrepsize, length(keys(ev.feats))))
    return FeatureSource(postags,xpostags,feats)
end

createcolvecs (generic function with 1 method)

In [25]:
fs = createcolvecs(corpus,ev)

FeatureSource([-0.542853 -0.992666 … 1.6983 -2.17757; 0.584793 1.14789 … 0.609551 -2.53964; … ; -0.825722 -2.10416 … 1.05649 -0.0918268; -2.90343 -0.18003 … -0.493267 1.06719], [1.46431 -0.123083 … 0.537261 0.982212; 0.260871 0.318797 … 0.798075 0.462872; … ; 1.29726 0.789473 … -0.589126 0.00480883; -1.61916 1.94446 … -1.57499 -1.55228], [0.819346 0.699932 … -0.915596 -1.791; 0.504342 -2.51908 … -0.0451559 0.201541; … ; 0.493422 0.875163 … -0.962536 1.63154; 0.395833 0.817469 … -1.82224 0.441632])

In [12]:
fs.xpostags[:,s.xpostag[1]]

32-element Array{Float64,1}:
 -0.12308284078752839 
  0.3187970252733201  
 -0.7910831402314742  
  0.4421427803274385  
 -1.0104910883715064  
  0.4859593864808637  
  0.07200507807828677 
 -1.2444991825818057  
  0.7321681826266581  
 -1.1034372169774     
  1.9233445877702353  
  1.3775612444949075  
 -1.867527226614093   
  ⋮                   
 -0.050779830784901565
 -0.7361552384181024  
 -0.9948592466621384  
 -0.27768412415298127 
 -0.4450002990337082  
  0.9776749351851451  
 -0.7948204573786628  
  0.8486637089024585  
 -1.4446783561804906  
 -0.6733567529686127  
  0.7894731805379925  
  1.9444589221380262  

In [26]:
# fs: feature source, feats: features of a word, s.feats[i]
function sumfeats(feats, fs)
    sum = zeros(length(fs.feats[:,1]))
    for i in 1:length(feats)
        sum .+= fs.feats[:,feats[i]]
    end
    return sum
end

sumfeats (generic function with 1 method)

In [14]:
# Version where we concat xpostag, postag, feats after creating fvec, bvec
function createfeatvec!(corpus, fs)
    for sent in corpus
        for i in 1:length(sent)
            push!(sent.cavec, vcat(sent.wvec[i], sent.fvec[i], sent.bvec[i], fs.postags[:,sent.postag[i]], fs.xpostags[:,sent.xpostag[i]], sumfeats(sent.feats[i],fs)))
        end
    end
end

createfeatvec! (generic function with 1 method)

In [27]:
words, sents, maxwordlen, maxsentlen = maptoint(corpus, v)

(Array{Int64,1}[[1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1], [1, 1, 1]  …  [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]], Array{Int64,1}[[3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 9], [14, 15, 16, 17, 18, 19, 15, 20, 21, 22, 23, 24, 25, 26, 9], [27, 28, 9], [29, 30, 31, 9], [32, 33, 34, 9], [35, 36, 37, 9], [38, 39], [40, 41, 9], [42, 43, 44, 9]  …  [13779, 13780, 1124, 73, 13781, 13782, 9633, 9], [13783, 13784, 15, 13785, 13786, 13787, 606, 607, 99, 39], [13788, 9628, 1915, 215, 9], [13789, 15, 854, 658, 4761, 13790, 9], [13791, 13792, 9], [5308, 13793, 6209, 146, 9], [8698, 5901, 160, 13794, 7669, 3910, 13795, 15, 13796, 13797, 1161, 941, 13

In [20]:
words

13808-element Array{Array{Int64,1},1}:
 [1, 1, 1]                        
 [1, 1, 1, 1]                     
 [1, 1, 1, 1, 1]                  
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1]                           
 [1, 1, 1, 1, 1, 1, 1, 1]         
 [1, 1, 1, 1, 1, 1, 1, 1, 1]      
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
 [1]                              
 [1, 1, 1]                        
 [1, 1, 1]                        
 [1, 1, 1, 1, 1, 1]               
 [1, 1, 1, 1, 1, 1]               
 ⋮                                
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1]         
 [1, 1, 1, 1, 1, 1]               
 [1, 1, 1, 1, 1, 1, 1, 1]         
 [1, 1, 1, 1, 1, 1, 1]            
 [1, 1, 1, 1, 1, 1, 1, 1]         
 [1, 1, 1, 1, 1, 1, 1, 1]         
 [1, 1, 1, 1, 1, 1, 1, 1]         
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1]      
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1]               

In [21]:
sents

3685-element Array{Array{Int64,1},1}:
 [3, 4, 5, 6, 7, 8, 9]                                                                                            
 [10, 11, 12, 13, 9]                                                                                              
 [14, 15, 16, 17, 18, 19, 15, 20, 21, 22, 23, 24, 25, 26, 9]                                                      
 [27, 28, 9]                                                                                                      
 [29, 30, 31, 9]                                                                                                  
 [32, 33, 34, 9]                                                                                                  
 [35, 36, 37, 9]                                                                                                  
 [38, 39]                                                                                                         
 [40, 41, 9]                              

In [116]:
wembed_extension = []
id = 1

for w in (v.sosword, v.eosword)
    push!(wembed_extension, ones(tagrepsize*3)) # 3 because 3*32 = xpostag, upostag, featssum
    id += 1
end

for (s, is) in zip(corpus, sents) # iterate sentences
    for i in 1:length(s.word) # iterate each word in a sentence
        if id == is[i]
            push!(wembed_extension, vcat(fs.postags[:,s.postag[i]], fs.xpostags[:,s.xpostag[i]], sumfeats(s.feats[i],fs)))
            id += 1
        end
    end
end

In [121]:
extended_wembed = vcat(wembed,hcat(wembed_extension...)) # concat version(446X13808) = wembed + wembed_extension(postag,xpostag,feat)

446×13808 Array{Float64,2}:
 -2.69957e-40  -2.83062e-43  -1.26117e-44  …  -0.0          -0.0        
 -0.994933     -0.00016526   -0.00206863      -0.000105809  -0.00137349 
 -0.779651     -0.0127146    -0.348087        -0.999725     -0.944697   
  3.42825e-12   1.26735e-16   1.50556e-17      9.45362e-17   7.83054e-20
 -2.6293e-6    -2.98281e-7   -1.57079e-9      -1.19129e-8   -3.12225e-9 
 -4.38526e-11  -1.21117e-15  -9.77352e-9   …  -1.97869e-8   -9.47691e-9 
 -0.0562993     0.736891      0.959822         1.0           0.994466   
  9.73939e-5    0.00406111    0.737888         9.75312e-6    0.611455   
 -0.668291     -4.47e-5      -1.57749e-5       3.2754e-7     0.00037853 
 -0.734724      2.41225e-6    1.5556e-5        1.97777e-9    6.31105e-10
  1.01045e-8    1.42305e-12   1.76551e-16  …   1.3879e-13    1.5978e-15 
  0.999281      0.999902      0.99995          1.0           0.999992   
 -2.118e-9     -5.35587e-13  -2.35052e-11     -2.19408e-8   -2.90514e-10
  ⋮                    

In [123]:
forw, back = wordlstm(wmodel, wdata, wmask, extended_wembed)

DimensionMismatch: DimensionMismatch("matrix A has dimensions (1200,650), matrix B has dimensions (746,101)")

In [124]:
czero = fill!(similar(bchar(wmodel), div(length(bchar(wmodel)), 4), length(wdata[1])), 0)

350×101 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [125]:
weight = wforw(wmodel)

1200×650 Array{Float32,2}:
  1.47578       0.055072   …  -0.115737   -0.2823     -1.58338   
 -1.95253      -0.402249      -0.873121   -0.221281   -0.667834  
 -0.526001     -0.0564435     -3.00089    -0.11858     0.565556  
  0.649006      0.707102      -0.23763     1.02644     1.56527   
 -1.71965      -0.116886      -0.809308   -0.410948    0.495027  
 -0.468197     -0.0351451  …  -0.795027    0.0951103  -1.66223   
  0.0470323    -0.63828        0.326873    0.257464    1.14677   
  0.177008      0.118655       0.899415    0.368408   -0.00539597
 -0.491867      0.152639      -0.381211   -0.613777   -2.16936   
 -2.02335       4.96995        0.668898    0.0449617  -1.8398    
  1.16926       0.588331   …   2.39633    -0.585489    1.8399    
 -2.2329        0.439566      -0.13345     0.398512    0.121094  
 -0.304711     -0.375472      -0.0561966  -0.347212    0.251312  
  ⋮                        ⋱                                     
  4.48859       1.98822       -0.899976   -0.2449